<a href="https://colab.research.google.com/github/omkenge/cookbook/blob/apollo/examples/Apollo_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompting with an Apollo 11 transcript

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Apollo_11.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

This notebook provides a quick example of how to prompt Gemini using a text file. In this case, you'll use a 400 page transcript from [Apollo 11](https://www.nasa.gov/history/alsj/a11/a11trans.html).

In [2]:
%pip install -U -q "google-genai>=1.7.0"

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [4]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

Download the transcript.

In [5]:
!wget https://storage.googleapis.com/generativeai-downloads/data/a11.txt

--2025-04-05 08:12:28--  https://storage.googleapis.com/generativeai-downloads/data/a11.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.207, 74.125.197.207, 74.125.135.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847790 (828K) [text/plain]
Saving to: ‘a11.txt’

a11.txt             100%[===================>] 827.92K  --.-KB/s    in 0.007s  

2025-04-05 08:12:29 (120 MB/s) - ‘a11.txt’ saved [847790/847790]



Upload the file using the File API so its easier to pass it to the model later on.

In [6]:
text_file_name = "a11.txt"
print(f"Uploading file...")
text_file = client.files.upload(file=text_file_name)
print(f"Completed upload: {text_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/8m6sosor9tvy


## Generate Content

After the file has been uploaded, you can make `client.models.generate_content` requests that reference the File API URI. Then you will ask the model to find a few lighthearted moments.

In [7]:
from IPython.display import display, Markdown
prompt = "Find four lighthearted moments in this text file."

MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

response = client.models.generate_content(
  model=f"models/{MODEL_ID}",
  contents=[
   prompt,
   text_file
  ],
  config={
   "httpOptions": {"timeout": 10 * 60 * 1000}
  }
)

display(Markdown(response.text))

Here are four lighthearted moments from the Apollo 11 transcript:

1.  **00 00 05 35 CDR: "You sure sound clear down there, Bruce. Sounds like you're sitting in your living room."** This comment is light and friendly, acknowledging the quality of communication.
2.  **00 00 05 39 CC: "Oh, thank you. You all are coming through beautifully, too."** This is a polite and humorous response to the previous comment.
3.  **00 00 54 13 CMP: "And tell Glenn Parker down at the Cape that he lucked out."**  This is a humorous, informal message sent to someone on the ground, suggesting they were fortunate in some way.
4. **00 02 28 45 CMP: "I wanted to be 18 or 20 pounds above nominal, babe." 00 02 28 49 CC: "Sorry about that."**  This exchange suggests a light tone of self-deprecating humor to a mistake made.


## Delete File

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [8]:
client.files.delete(name=text_file.name)

DeleteFileResponse()

## Learning more

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Learn more about the [File API](../quickstarts/File_API.ipynb) here.